In [1]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore
from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot
import bk.signal
import bk.stats

import os


import astropy.stats as apy
plt.rcParams['svg.fonttype'] = 'none'

In [2]:
bk.load.current_session_linux()
neurons, metadata = bk.load.spikes()
state = bk.load.states()['Rem']
low = 4
high = 12
nbins = 1000
channel = bk.load.ripple_channel()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset-Light/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [8]:
def fr(spikes,state):
    return(len(spikes.restrict(state))/state.tot_length(time_units='s'))

def mod_stats(spikes,phases,nbins):
    phases_signal_distrib, _ = np.histogram(phases.values,nbins,density=True)
    phases_signal_distrib = phases_signal_distrib*nbins / np.sum(phases_signal_distrib)


    neuron_phase = phases.realign(spikes)
    
    stats = {
        'pvalue':np.nan,
        'kappa':np.nan,
        'MRL':np.nan
    }
    
    if len(neuron_phase) == 0: return stats

    phases_distribution, bin_p = np.histogram(neuron_phase.values, nbins) 
    phases_distribution = phases_distribution/phases_signal_distrib

    bin_p = np.convolve(bin_p, [.5, .5], 'same')[1::]


    stats['pvalue'] = bk.stats.rayleigh(bin_p,weights = phases_distribution)
    stats['kappa'] = bk.compute.concentration(neuron_phase.values)
    stats['MRL'] = bk.compute.mean_resultant_length(bin_p,weights = phases_distribution)

    return bin_p,phases_distribution,stats
    
def phase_mod(neurons,metadata,state,low,high,nbins = 1000,channel=None):
    lfp = bk.load.lfp_in_intervals(channel,state)
    lfp_filt = bk.signal.passband(lfp,low,high,1250,2)
    power, phase = bk.signal.hilbert(lfp_filt)

    stats = {'fr_rem': [],
             'pvalue': [],
             'kappa': [],
             'MRL': [],
             'modulated':[]}
    for i,n in tqdm(enumerate(neurons)):
        bin_p, phases_distributions,stat = mod_stats(n.restrict(state),phase,nbins)
        stats['fr_rem'].append(fr(n,state))
        for k, s in stat.items():
            stats[k].append(s)
        if (stat['pvalue'] < 0.01) and stat['MRL'] > 0.04:
            stats['modulated'].append(True)
        else:
            stats['modulated'].append(False)

    phase_lock = pd.DataFrame({'FR_Rem': stats['fr_rem'],
                               'MRL': stats['MRL'],
                               'Kappa': stats['kappa'],
                               'pValue': stats['pvalue'],
                               'modulated': stats['modulated']})
    
        
    return bin_p,phases_distributions,phase_lock

In [9]:
bin_p,phases_distributions,phase_lock = phase_mod(neurons,metadata,state,4,12,1000,bk.load.ripple_channel())

252it [00:09, 26.10it/s]


In [10]:
phase_lock

,FR_Rem,MRL,Kappa,pValue,modulated
0,0.730072,0.027169,0.249840,0.278427,False
1,0.799241,0.066323,0.470502,0.000233,True
2,0.661324,0.083081,0.889105,0.000020,True
3,1.222691,0.067302,0.453353,0.000002,True
4,2.689582,0.032987,0.558136,0.000977,False
...,...,...,...,...,...
247,0.800506,0.008380,0.296848,0.875158,False
248,0.062843,0.015800,0.212561,0.963510,False
249,0.092366,0.048025,0.360928,0.603532,False
250,0.105863,0.031857,0.522764,0.775465,False
